# Codigo para geração do indice invertido

In [2]:
import pandas as pd
import math
import nltk
from nltk import bigrams
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import numpy as np
import string
from scipy import sparse
import scipy.sparse as sps

In [3]:
class WordInfo:
    'Classe que guarda informações sobre a palavra, como documentos em que ocorre, tf em cada documento e calculo de idf'

    def __init__(self, word) : # Contrutor da classe recebe apenas a palavra em questão
        self.word = word
        self.idf = 0
        self.docs = {} # Dicionario que mapeia doc_id ao tf da palavra

    def found(self, doc_id) : # Metodo que realiza a contagem do tf da palavra
        if(doc_id in self.docs) : # Se o doc_id está mapeado, incrementa-o
            self.docs[doc_id] += 1
        else :
            self.docs[doc_id] = 1 # Caso contrario, define-o como 1
            
    def calculateIDF(self, totaldocs) : # Metodo de calculo do idf, recebendo o total de documentos
        df = len(self.docs) # Numero de documentos em que a palavra ocorre 
        if (df > 0) :
            self.idf = math.log(((totaldocs + 1)/df))
            
    def getIds(self) : # Metodo que retorna todos os doc_ids em que a palavra ocorre
        return list(self.docs.keys())
    
    def getTf(self, doc_id) : # Metodo que retorna o tf da palavra em um documento, ou 0 caso não ocorra
        return self.docs.get(doc_id, 0)

In [4]:
tabela = pd.read_csv("estadao_noticias_eleicao.csv", encoding="utf-8")
tabela.fillna('', inplace=True) # Preenchendo os campos vazios da tabela com ''

In [5]:
mapa = {} # Dicionario/Mapa que representa o indice invertido

In [6]:
documentos = {} # Dicionario que guarda os documentos e seus tamanhos

In [7]:
for index, linha in tabela.iterrows() : # Iterando sobre as noticias no arquivo
    id = linha['idNoticia'] # Recuperando o idNoticia da noticia atual
    texto = nltk.wordpunct_tokenize(linha['titulo'] + ' ' + linha['subTitulo'] + ' ' + linha['conteudo']) # Tokenização do texto
    documentos[id] = len(texto) # Mapeamento do documento com seu tamanho
    for palavra in texto: # Iterando sobre as palavras na Noticia
        if (palavra not in string.punctuation): # Só ocorre o mapeamento se a palavra não é uma pontuação
            if (palavra.lower() not in mapa) : # Se a palavra ainda não foi mapeada, mapeia-a
                mapa[palavra.lower()] = WordInfo(palavra.lower())
            mapa[palavra.lower()].found(id) # Contabiliza a ocorrencia da palavra no documento atual

for k in mapa.keys() : # Laço para realizar o calculo dos idfs de cada palavra mapeada
    mapa[k].calculateIDF(len(documentos))

# Funções de Consultas Booleanas(And, Or e geral)

In [8]:
def searchAnd(palavra1, *palavras) :
    docs1 = mapa[palavra1.lower()].getIds() # Recupera todos as noticias em que "palavra1" ocorreu
    docs2 = mapa[palavra2.lower()].getIds() # Recupera todos as noticias em que "palavra2" ocorreu
    result = set() # Cria um conjunto vazio
    result.update(docs1) # Preenche o conjunto com os ids das noticias que "palavra1" aparece
    for palavra in palavras :
        docs = mapa[palavra.lower()].getIds()
        result = result.intersection(docs)
    return list(result)

In [9]:
def searchOr(palavra1, *palavras) :
    docs1 = mapa[palavra1.lower()].getIds()
    result = set() # Cria um conjunto vazio
    result.update(docs1) # Preenche o conjunto com os ids das noticias que "palavra1" aparece
    for palavra in palavras :
        docs = mapa[palavra.lower()].getIds()
        result.update(docs)
    return list(result)

In [10]:
def search(consulta) :
    partes = consulta.split(' ')
    if (len(partes) < 2) : # Se a consulta só tem uma palavra
        return mapa[partes[0].lower()].getIds() # Recupera os ids que a palavra aparece
    elif (partes[1].upper() == 'AND') : # Se é uma consulta AND
        return searchAnd(partes[0], partes[2]) # Chama a função de consulta AND
    elif (partes[1].upper() == 'OR') : # Se é uma consulta OR
        return searchOr(partes[0], partes[2]) # Chama a função de consulta OR

# Funções de Consultas Vetoriais

In [11]:
def busca_binaria(consulta) :
    palavras = consulta.split(' ') # Quebrando a consulta em palavras
    relevant_docs = set(mapa[palavras[0]].getIds()) # Inicia o conjunto de documentos relevantes
    # Laço que realiza interseção dos conjuntos, para mantes apenas documentos em que todas as palavras a consulta ocorre
    for i in range(1, len(palavras)) :
        relevant_docs = relevant_docs.intersection(set(mapa[palavras[i]].getIds()))
    # Como é busca binaria, apenas retorna os primeiros 5 documentos que contem todas as palavras
    return list(relevant_docs)[:5]

In [12]:
def busca_tf(consulta) :
    palavras = consulta.split(' ') # Quebrando a consulta em palavras
    relevant_docs = set(mapa[palavras[0]].getIds()) # Inicia o conjunto de documentos relevantes
    # Laço que realiza interseção dos conjuntos, para mantes apenas documentos em que todas as palavras a consulta ocorre
    for i in range(1, len(palavras)) :
        relevant_docs = relevant_docs.intersection(set(mapa[palavras[i]].getIds()))
    result = [] # Lista que será gerado o resultado
    for doc_id in relevant_docs : # Para cada documento relevante
        scores = [mapa[w].getTf(doc_id) for w in palavras] # Obtem os tfs de cada palavra da consulta
        # Cria uma tupla (score, doc_id) somando os tfs obtidos e coloca na lista
        score_id = (sum(scores), doc_id)
        result.append(score_id)
    # Ordena do maior para o menor, considerando apenas o primeiro elemento da tupla para ordenar (o score)
    result = sorted(result, reverse=True, key=lambda tup: tup[0]) 
    result = [t[1] for t in result] # A lista passa a ser apenas dos doc_ids
    return result[:5] # Retorna os 5 primeiros

In [13]:
def busca_tfidf(consulta) :
    palavras = consulta.split(' ') # Quebrando a consulta em palavras
    relevant_docs = set(mapa[palavras[0]].getIds()) # Inicia o conjunto de documentos relevantes
    # Laço que realiza interseção dos conjuntos, para mantes apenas documentos em que todas as palavras a consulta ocorre
    for i in range(1, len(palavras)) :
        relevant_docs = relevant_docs.intersection(set(mapa[palavras[i]].getIds()))
    result = [] # Lista que será gerado o resultado
    for doc_id in relevant_docs : # Para cada documento relevante
        scores = [] # Lista para guardar os scores de cada palavra da consulta
        for w in palavras : # Calcula tf * idf de cada palavra da consulta e adiciona na lista
            m_palavra = mapa[w]
            w_score = m_palavra.getTf(doc_id) * m_palavra.idf
            scores.append(w_score)
        # Cria uma tupla (score, doc_id) somando os resultados obtidos e coloca na lista
        score_id = (sum(scores), doc_id)
        result.append(score_id)
    # Ordena do maior para o menor, considerando apenas o primeiro elemento da tupla para ordenar (o score)
    result = sorted(result, reverse=True, key=lambda tup: tup[0])
    result = [t[1] for t in result] # A lista passa a ser apenas dos doc_ids
    return result[:5] # Retorna os 5 primeiros

In [14]:
def busca_bm25(consulta) :
    k = 6 # 6 foi o valor de K que maximizou o resultado da função mapk
    palavras = consulta.split(' ') # Quebrando a consulta em palavras
    relevant_docs = set(mapa[palavras[0]].getIds()) # Inicia o conjunto de documentos relevantes
    # Laço que realiza interseção dos conjuntos, para mantes apenas documentos em que todas as palavras a consulta ocorre
    for i in range(1, len(palavras)) :
        relevant_docs = relevant_docs.intersection(set(mapa[palavras[i]].getIds()))
    result = [] # Lista que será gerado o resultado
    for doc_id in relevant_docs : # Para cada documento relevante
        scores = [] # Lista para guardar os scores de cada palavra da consulta
        for w in palavras : # Calcula tf*(k+1)/(tf + k) * idf de cada palavra da consulta e adiciona na lista
            m_palavra = mapa[w]
            tf = m_palavra.getTf(doc_id)
            w_score = (tf*(k+1))/(tf + k) * m_palavra.idf
            scores.append(w_score)
        # Cria uma tupla (score, doc_id) somando os resultados obtidos e coloca na lista
        score_id = (sum(scores), doc_id)
        result.append(score_id)
    # Ordena do maior para o menor, considerando apenas o primeiro elemento da tupla para ordenar (o score)
    result = sorted(result, reverse=True, key=lambda tup: tup[0])
    result = [t[1] for t in result] # A lista passa a ser apenas dos doc_ids
    return result[:5] # Retorna os 5 primeiros

# Expansão de Consulta

Função de criação da matrix de termos-termos

In [15]:
def co_occurrence_matrix(corpus):
    vocab = set(corpus)
    vocab = list(vocab)
    n = len(vocab)
   
    vocab_to_index = {word:i for i, word in enumerate(vocab)}
    
    bi_grams = list(bigrams(corpus))

    bigram_freq = nltk.FreqDist(bi_grams).most_common(len(bi_grams))

    I=list()
    J=list()
    V=list()
    
    for bigram in bigram_freq:
        current = bigram[0][1]
        previous = bigram[0][0]
        count = bigram[1]

        I.append(vocab_to_index[previous])
        J.append(vocab_to_index[current])
        V.append(count)
        
    co_occurrence_matrix = sparse.coo_matrix((V,(I,J)), shape=(n,n))

    return co_occurrence_matrix, vocab_to_index

### Montagem da matrix

In [16]:
content = tabela.titulo + " " + tabela.subTitulo + " " +  tabela.conteudo

Remoção de pontuação

In [17]:
tokenizer = RegexpTokenizer(r'\w+')
tokens_lists = content.apply(lambda text: tokenizer.tokenize(text.lower()))

Remoção de stopwords

In [19]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\SauloSamuel\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [20]:
stopword_ = stopwords.words('portuguese')
filtered_tokens = tokens_lists.apply(lambda tokens: [token for token in tokens if token not in stopword_])

Transformando lista de listas em uma lista

In [21]:
tokens = [token for tokens_list in filtered_tokens for token in tokens_list]

In [22]:
matrix, vocab = co_occurrence_matrix(tokens)

### Consulta de frequência de bigramas

In [23]:
consultable_matrix = matrix.tocsr()

Função de consulta

In [24]:
def consult_frequency(w1, w2):
    return(consultable_matrix[vocab[w1],vocab[w2]])

### Expandindo a consulta

Função que conta a coocorrência de uma palavra passada com todas as outras palavras do corpud e recupera as 3 que mais ocorrem com a palavra passada

In [25]:
def top3_bigram(palavra) :
    bigram_freq = []
    for key in vocab.keys() :
        bigram_freq.append((consult_frequency(palavra, key), key))
    bigram_freq = sorted(bigram_freq, reverse=True)[:3]
    return [t[1] for t in bigram_freq if t[0] > 0 ]

Função que expande a consulta, recebendo um único termo e realizando uma consulta "OR" entre o termo passado e as 3 palavras que mais ocorrem com ele 

In [26]:
def expandedSearch(word) :
    top3 = top3_bigram(word)
    return searchOr(word, *top3)

# Exemplos

### Escolha livremente três termos de consulta e responda o seguinte:
##### Quais os termos retornados para a expansão de cada consulta?

'governo': 'federal', 'dilma' e 'estado'

In [27]:
top3_bigram('governo')

['federal', 'dilma', 'estado']

'projeto': 'lei', 'politico' e 'poder'

In [28]:
top3_bigram('projeto')

['lei', 'político', 'poder']

'mandato': 'dilma', 'presidente' e 'deputado'

In [29]:
top3_bigram('mandato')

['dilma', 'presidente', 'deputado']

###### Você acha que esses termos são de fato relacionados com a consulta original? Justifique.

Sim, dado o contexto de noticias politicas em questão, as palavras retornadas se mostram muito relacionadas aos termos originais, é comum nesse universo "governo federal", "projeto de lei"...
Alguns são claramente bigramas ("governo dilma"), alguns tem uma stopword entre eles ("governo do estado").
O bigrama que parece mais distante é 'projeto' e 'poder', ainda assim são relacionados.

###### Compare os documentos retornados para a consulta original com a consulta expandida. Quais resultados você acha que melhor capturam a necessidade de informação do usuário? Por que?

Como pode-se ver abaixo, a consulta expandida chega a retornar mais de 5x mais documentos que a consulta original, que já é um número bem grande de documentos, isso sem dúvida faz a consulta original melhor, porque normalmente o usuário tenta olhar cerca de 10 resultados.

In [30]:
print('GOVERNO - Consulta original: {} | Consulta expandida: {}'.format(len(search('governo')), len(expandedSearch('governo'))))

GOVERNO - Consulta original: 4983 | Consulta expandida: 7142


In [31]:
print('PROJETO - Consulta original: {} | Consulta expandida: {}'.format(len(search('projeto')), len(expandedSearch('projeto'))))

PROJETO - Consulta original: 1156 | Consulta expandida: 4773


In [32]:
print('MANDATO - Consulta original: {} | Consulta expandida: {}'.format(len(search('mandato')), len(expandedSearch('mandato'))))

MANDATO - Consulta original: 1219 | Consulta expandida: 6442


In [33]:
print('5 dos documentos retornados apenas na consulta expandida para GOVERNO:\n{}'.format(list(set(expandedSearch('governo'))-set(search('governo')))[:5]))

5 dos documentos retornados apenas na consulta expandida para GOVERNO:
[8194, 8195, 8196, 8, 9]


In [34]:
print('5 dos documentos retornados apenas na consulta expandida para PROJETO:\n{}'.format(list(set(expandedSearch('projeto'))-set(search('projeto')))[:5]))

5 dos documentos retornados apenas na consulta expandida para PROJETO:
[1, 2, 3, 13, 15]


In [35]:
print('5 dos documentos retornados apenas na consulta expandida para MANDATO:\n{}'.format(list(set(expandedSearch('mandato'))-set(search('mandato')))[:5]))

5 dos documentos retornados apenas na consulta expandida para MANDATO:
[9, 12, 13, 15, 17]


Na minha opinião, alguns documentos que são retornados apenas pela consulta expandida não são nada adequados a consulta original (a maioria dos que olhei, pelo menos), dada a naturesa dos dados, todos eles estão ligados a política, para 'governo' e 'mandato' não acho que nenhum dos documentos são relevantes, 'projeto' tem os documentos 2 e 3 relevantes

###### A expansão de consultas é mais adequada para melhorar o recall ou o precision? Por que?

Recall, uma vez que embora ela vá retornar documentos relevantes que eventualmente não contém o termo original da consulta, ela retornará outros domumentos que contém os termos da espansão que não são relevantes para o usuário, é um grande aumento no número de documentos retornados para retornar mais documentos relevantes, mas muito deles não serão relevantes, diminuindo a precisão e aumentando o recall